### 설문 조사 데이터를 오버샘플링 하기 위한 코드
- 샘플링 수를 늘려보자 !

In [41]:
import pandas as pd
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

# 데이터 로드
original_data = pd.read_csv('./survey.csv')

# 데이터 스케일링 및 클러스터링
scaler = StandardScaler()
y_scaled = scaler.fit_transform(original_data[['work', 'edu', 'free_time', 'health', 'chores']])

kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(y_scaled)

# 범주형 데이터 원-핫 인코딩 및 특성 분리
X_encoded = pd.get_dummies(original_data.drop(['work', 'edu', 'free_time', 'health', 'chores'], axis=1))
X = pd.concat([X_encoded, original_data[['age', 'gender']]], axis=1)

# SMOTE 적용 (연속형 데이터 포함)
n_neighbors = max(min(2, len(X) - 1), 1)  # 최소 1, 최대 len(X)-1 사이
smote = SMOTE(random_state=42, k_neighbors=n_neighbors)
X_resampled, clusters_resampled = smote.fit_resample(X, clusters)

# 연속형 데이터 복원: 각 클러스터의 중심을 사용하여 연속형 데이터를 복원
cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)
oversampled_y = [cluster_centers[cluster] for cluster in clusters_resampled]

# 오버샘플링된 데이터 프레임 생성
oversampled_df = pd.DataFrame(X_resampled, columns=X.columns)
oversampled_df[['work', 'edu', 'free_time', 'health', 'chores']] = pd.DataFrame(oversampled_y, columns=['work', 'edu', 'free_time', 'health', 'chores'])

# 결과 저장 및 출력
oversampled_df.to_csv('./survey_oversampled.csv', index=False)
print("오버샘플링 데이터 저장 완료: survey_oversampled.csv")


개선된 오버샘플링 데이터 저장 완료: survey_oversampled.csv


In [42]:
import pandas as pd

# 오버샘플링된 데이터 로드
oversampled_data = pd.read_csv('./survey_oversampled.csv')

# 'job' 관련 열 추출 및 변환
job_columns = [col for col in oversampled_data.columns if col.startswith('job_')]
oversampled_data['job'] = oversampled_data[job_columns].idxmax(axis=1).apply(lambda x: x.split('_')[1] if pd.notna(x) else None)

# 'mbti' 관련 열 추출 및 변환
mbti_columns = [col for col in oversampled_data.columns if col.startswith('mbti_')]
oversampled_data['mbti'] = oversampled_data[mbti_columns].idxmax(axis=1).apply(lambda x: x.split('_')[1] if pd.notna(x) else None)

# 불필요한 원-핫 인코딩 열 제거
oversampled_data.drop(columns=job_columns + mbti_columns, inplace=True)

# 결과 저장
oversampled_data.to_csv('./survey_oversampled_fixed.csv', index=False)

print("데이터 업데이트 완료: survey_oversampled_fixed.csv")


데이터 업데이트 완료: survey_oversampled_fixed.csv


In [54]:
# 마지막으로 value 수정
fix_data = pd.read_csv('./survey_oversampled_fixed.csv')

# 불필요한 column들 제거
fix_data.drop(columns=['age.1','gender.1'], inplace=True)

# 'job' cloumn의 'job' 값을 'job_seeker'로 수정
fix_data['job'] = fix_data['job'].map(
    lambda x: 'job_seeker' if x == 'job' else x
    )

# 열 순서 재배열
columns_ordered = ['age', 'gender', 'job', 'mbti', 'edu', 'free_time', 'health', 'chores']
fix_data = fix_data[columns_ordered]

In [55]:
fix_data

,age,gender,job,mbti,edu,free_time,health,chores
0,27,0,job_seeker,INTJ,0.833333,1.000000,1.000000,0.416667
1,28,0,job_seeker,INTJ,0.666667,0.416667,0.583333,0.500000
2,28,1,senior,ESFP,0.720588,0.867647,0.838235,0.544118
3,23,0,senior,INTP,0.666667,0.416667,0.583333,0.500000
4,27,0,job_seeker,ISTP,0.666667,0.416667,0.583333,0.500000
5,29,0,job_seeker,ESTP,0.720588,0.867647,0.838235,0.544118
6,29,0,senior,INTP,0.833333,1.000000,1.000000,0.416667
7,27,0,job_seeker,INTP,0.666667,0.416667,0.583333,0.500000
8,22,1,senior,ISTP,0.720588,0.867647,0.838235,0.544118
9,27,0,job_seeker,ENTP,0.720588,0.867647,0.838235,0.544118
